In [ ]:
import networkx as nx
import heapq

In [ ]:
G = nx.DiGraph()
edges = [(1, 2, 10),
         (2, 3, 1),
         (3, 4, 10),
         (1, 9, 20),
         (9, 2, 1),
         (2, 6, 1),
         (6, 8, 1),
         (8, 9, 1),
         (2, 8, 3),
         (8, 5, 15),
         (3, 5, 18),
         (5, 4, 1)]

# A 1
# B 2
# C 3
# D 4
# E 5
# F 6
# G 7
# H 8
# I 9

G.add_weighted_edges_from(edges)

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
# partial dijkstra
def dijkstra(graph, src, dest):

  if src ==  dest:
    return 0

  heap = [(0, src)]
  shortest_path = {node: float('inf') for node in graph}
  shortest_path[src] = 0

  while heap:
    cost, node = heapq.heappop(heap)
    if node == dest:
      # print(shortest_path)
      return shortest_path[dest]

    if cost > shortest_path[node]:
      continue

    for neighbor, data in graph[node].items():
      new_cost = cost + data['weight']

      if new_cost < shortest_path[neighbor]:
        shortest_path[neighbor] = new_cost
        heapq.heappush(heap, (new_cost, neighbor))

  return float('inf')
 

In [ ]:
def reverse(graph):
  Gr = nx.DiGraph()
  # for node in graph.nodes():
  #   Gr.add_node(node)

  Gr.add_edges_from((v,u,d) for u,v,d in graph.edges(data=True))
  return Gr

In [ ]:
GR = reverse(G)

In [ ]:
nx.draw(GR, with_labels=True)

In [ ]:
dist = {}
isSettled = {}
PQ = []
parent = {}

for node in GR.nodes():
  dist[node] = float('inf')
  isSettled[node] = False
  parent[node] = None

In [ ]:
def ConstructPartialSPT(graph, v):
  global dist, isSettled, PQ, parent

  if isSettled[v]:
    return dist[v]

  while PQ:
    cost, node = heapq.heappop(PQ)

    if cost > dist[node]:
      continue

    if not isSettled[node]:
      isSettled[node] = True

      if node == v:
        return dist[v]      
      
      for neighbor, data in graph[node].items():

        if not isSettled[neighbor]:        
          new_cost = cost + data['weight']

          if new_cost < dist[neighbor]:
            dist[neighbor] = new_cost
            parent[neighbor] = node
            heapq.heappush(PQ, (new_cost, neighbor))
        
  return float('inf')

In [ ]:
ConstructPartialSPT(GR, 4, 1)
dist